In [ ]:
!pip install -q efficientnet

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pathlib
import imageio
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 
import plotly.express as px
import os
import cv2
from PIL import Image
import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from efficientnet.tfkeras import EfficientNetB1,EfficientNetB2, EfficientNetB3, EfficientNetB4

In [ ]:
input_dir = "../input/cassava-leaf-disease-classification"

train_images_path = os.path.join(input_dir,"train_images")
test_images_path = os.path.join(input_dir,'test_images')
train = pd.read_csv('../input/cassava-leaf-disease-classification/train.csv')

In [ ]:
image_list = train['image_id'].to_list()
label_list = train['label'].to_list()

In [ ]:
import json

with open("../input/cassava-leaf-disease-classification/label_num_to_disease_map.json") as f:
    class_mapping = json.load(f)

class_mapping2 ={int(k):v for k,v in class_mapping.items()}

class_mapping2

In [ ]:
BATCH_SIZE =8 #Mini-Batch Gradient Descent
STEPS_PER_EPOCH = len(train)*0.8 / BATCH_SIZE
VALIDATION_STEPS = len(train)*0.2 / BATCH_SIZE
EPOCHS = 1
TARGET_SIZE = 512
IMAGE_SIZE = [512, 512]

In [ ]:
train.label = train.label.astype('str')

train_datagen = ImageDataGenerator(validation_split = 0.2,
                                     rotation_range = 45,
                                     zoom_range = 0.3,
                                     horizontal_flip = True,
                                     vertical_flip = True,
                                     fill_mode = 'nearest',
                                     shear_range = 0.1,
                                     height_shift_range = 0.1,
                                     width_shift_range = 0.1,
                                     featurewise_center = True,
                                     featurewise_std_normalization = True)

train_generator = train_datagen.flow_from_dataframe(train,
                         directory = os.path.join('../input/cassava-leaf-disease-classification/train_images'),
                         subset = "training",
                         x_col = "image_id",
                         y_col = "label",
                         target_size = (TARGET_SIZE, TARGET_SIZE),
                         batch_size = BATCH_SIZE,
                         class_mode = "sparse",
                         shuffle= True)


validation_datagen = ImageDataGenerator(validation_split = 0.2)

validation_generator = validation_datagen.flow_from_dataframe(train,
                         directory = os.path.join('../input/cassava-leaf-disease-classification/train_images'),
                         subset = "validation",
                         x_col = "image_id",
                         y_col = "label",
                         target_size = (TARGET_SIZE, TARGET_SIZE),
                         batch_size = BATCH_SIZE,
                         class_mode = "sparse")

In [ ]:
def create_model():
    conv_base = EfficientNetB2(include_top=False, input_tensor=None,
                               pooling=None, input_shape=(TARGET_SIZE, TARGET_SIZE, 3))
                               
    model = conv_base.output
    model = layers.GlobalAveragePooling2D()(model)
    model = layers.Dense(5, activation = "softmax")(model)
    model = models.Model(conv_base.input, model)

    model.compile(optimizer = Adam(lr = 0.001),
                  loss = "sparse_categorical_crossentropy",
                  metrics = ["acc"])
    return model

In [ ]:
model = create_model()
model.summary()

In [ ]:
model_save = ModelCheckpoint('./efficb1best.h5', 
                              save_best_only = True, 
                              save_weights_only = True,
                              monitor = 'val_loss', 
                              mode = 'min', verbose = 1)
early_stop = EarlyStopping(monitor = 'val_loss', min_delta = 0.001, 
                            patience = 5, mode = 'min', verbose = 1,
                            restore_best_weights = True)
reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.3, 
                               patience = 2, min_delta = 0.001, 
                               mode = 'min', verbose = 1) #reduced learning rate


history = model.fit(
     train_generator,
     steps_per_epoch = STEPS_PER_EPOCH,
     epochs = EPOCHS,
     validation_data = validation_generator,
     validation_steps = VALIDATION_STEPS,
     callbacks = [model_save, early_stop, reduce_lr])

In [ ]:
 model.save('./efficb1.h5')